In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
# Selenium para establecer la configuración del driver
# -----------------------------------------------------------------------
from selenium import webdriver

# Para generar una barra de proceso en los bucles for
# -----------------------------------------------------------------------
from tqdm import tqdm

# Para trabajar con ficheros
# -----------------------------------------------------------------------
import os

import re

In [ ]:
base_url_demanda = "https://apidatos.ree.es/es/datos/demanda/evolucion"
base_url_generacion = "https://apidatos.ree.es/es/datos/generacion/estructura-renovables"
headers = {'Accept': 'application/json'}

In [ ]:
def obtener_datos(tipo, anio, comunidad_id):
    base_url = base_url_demanda if tipo == 'demanda' else base_url_generacion
    fecha_inicio = f"{anio}-01-01"
    fecha_fin = f"{anio}-12-31"
    params = {'start_date': fecha_inicio,
            'end_date': fecha_fin,
            'time_trunc': 'month',
            'geo_limit': 'ccaa',
            'geo_ids': comunidad_id}
    response = requests.get(base_url, headers=headers, params=params)

In [ ]:
if response.status_code == 200:
        data = response.json()
        valores = data['included'][0]['attributes']['values']
        df = pd.DataFrame(valores)
        df['Tipo'] = tipo
        df['Id_Comunidad']=comunidad_id
        return df
    else:
        print(f"Error {response.status_code} al solicitar {tipo} para {comunidad_id} en el año {anio}")
        return pd.DataFrame()